In [234]:
import pandas as pd

In [235]:
dfs = pd.read_html('https://www.basketball-reference.com/playoffs/NBA_2023.html')
df = dfs[20]  # the third table in the page and remove the last row

In [236]:
df.columns = df.columns.droplevel()

In [237]:
df = df.loc[:,['Team','W','L']]

In [238]:
df["Round Finished"] = df.apply(lambda row: 5 if row["W"] == 16 else
                                 4 if row["W"] >= 12 else
                                 3 if 8 <= row["W"] < 12 else
                                 2 if 4 <= row["W"] < 8 else
                                 1 if row["W"] < 4 and row["L"]>0 else
                                 -1, axis=1)
df = df.drop(['W','L'],axis=1)

In [239]:
new = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2023_standings.html')

In [240]:
east = new[0]
west = new[1]

In [241]:
west = west["Western Conference"].apply(lambda x: x.split("*")[0].strip()).str.split("(", expand=True)[0].str.strip()
east = east["Eastern Conference"].apply(lambda x: x.split("*")[0].strip()).str.split("(", expand=True)[0].str.strip()

In [242]:
teams = list(pd.concat([east,west]).reset_index(drop=True))

In [243]:
for value in teams:
    if value not in df['Team'].values:
        df.loc[len(df)] = { 'Team': value, 'Round Finished': -1}

In [244]:
df = df.sort_values('Round Finished',ascending = False).reset_index(drop = True)
df

,Team,Round Finished
0,Denver Nuggets,5
1,Miami Heat,4
2,Los Angeles Lakers,3
3,Boston Celtics,3
4,New York Knicks,2
5,Golden State Warriors,2
6,Philadelphia 76ers,2
7,Phoenix Suns,2
8,Los Angeles Clippers,1
9,Minnesota Timberwolves,1
